In [1]:
import csv
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib import colors as mcolors
import os
from datetime import datetime
import bisect 
import re

colors = dict(mcolors.BASE_COLORS, **mcolors.CSS4_COLORS)

with open(csvpath+"result.csv",newline='') as f:
    r = csv.reader(f)
    data = [line for line in r]
with open(csvpath+"result.csv",'w',newline='') as f:
    w = csv.writer(f)
    w.writerow(['doi','count_ref','ref','journal','1','2','3','4','5'])
    w.writerows(data)

In [2]:
def scanLog(csvlogpath, txt_to_search, txt_field, printit = False):
    with open(csvlogpath) as csvfile:
        print('SCANNING ' + csvlogpath)
        reader = csv.DictReader(csvfile)
        count = 0
        data = []
        
        for row in reader:
            is_ok = True
            for i in range(len(txt_field)):
                
                pattern = re.compile(txt_to_search[i])
                if pattern.match(row[txt_field[i]]):
                    is_ok = True
                else:
                    is_ok = False
                    break
            
            if is_ok:
                count += 1
                
                my_val = {}
                for i in range(len(txt_field)):
                    my_val[txt_field[i]] = row[txt_field[i]]
                #data.append(my_val)
                data.append(row)
            
        return {'count': count, 'value': data}
    
def plotLines(data,opt = {}, logit= False):
    for d in data:
        my_color = None
        if 'color' in opt:
            if d in opt['color']:
                my_color = opt['color'][d]

        my_width = None
        if 'width' in opt:
            if d in opt['width']:
                my_width = opt['width'][d]
        plt.plot(data[d]['x'],data[d]['y'], label=d, color=my_color, linewidth=my_width)
        if logit:
            plt.yscale('log')
            
        plt.xticks(data[d]['x'],[item.strftime('%Y-%m') for item in data[d]['x']])
    return plt

def plotPie(data,opt ={}, logit= False):
    
    #explode = (0.05, 0.05, 0.05, 0.05)
   #autopct='%1.1f%%' 
    fig1, ax1 = plt.subplots()
    ax1.pie(data['sizes'], labels=data['labels'])
    ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    return plt
    

In [9]:
def buildData(txt_list, txt_field_list, csvpath, not_inc = []):
    data = {'x':[],'y':[],'value':[]}
    for filename in os.listdir(csvpath):
        if filename.endswith(".csv") and filename not in not_inc : 
            csvlogpath = csvpath+filename
            scanner = scanLog(csvlogpath,txt_list, txt_field_list)
            date = filename.replace("oc-", "").replace(".csv", "")

            #date_val = datetime.strptime(date, '%Y-%m')
            date_val = "result"
            bisect.insort(data['x'],date_val) 
            ord_index = data['x'].index(date_val)
            data['y'].insert(ord_index,scanner['count'])
            data['value'].insert(ord_index,scanner['value'])
        else:
            continue
        
    return data

In [10]:
csvpath = '/Users/ivan.heibi/PhD/citation_fun/'
#TIME,REMOTE_ADDR,HTTP_USER_AGENT,HTTP_REFERER(from),HTTP_HOST,REQUEST_URI(to)

In [41]:
data = {}

In [42]:
data['journal'] = buildData(['.*'], ['4'], csvpath)

SCANNING /Users/ivan.heibi/PhD/citation_fun/result.csv


In [44]:
print(len(data['journal']['value'][0]))

6313


In [58]:
journals = {}
for elem in data['journal']['value'][0]:
    if elem['4'] in journals:
        journals[elem['4']] += 1
    else:
        journals[elem['4']] = 1

In [61]:
journals = {}
for elem in data['journal']['value'][0]:
    if elem['4'] in journals:
        journals[elem['4']].append(elem['3'])
    else:
        journals[elem['4']] = [elem['3']]

In [62]:
arr = []
for elem in journals:
    arr.append((elem,journals[elem]))


In [64]:
s_arr = sorted(arr, key=lambda tup: tup[1],reverse = True)

In [66]:
s_arr[301]

('The Journal Of Environmental Education',
 ['The Evolution Of Ee As Inferred From Citation Analysis',
  'A Bibliometric Analysis Of The Literature Of Environmental Education'])